<a href="https://colab.research.google.com/github/kimdonggyu2008/audio_synthesis/blob/main/soundgenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import librosa
from sound_preprocess import MinMaxNormaliser


In [ ]:
class SoundGenerator: #스펙트로그램에서 소리 생성
  def __init__(self,vae,hop_length): #초기값 선언
    self.vae=vae
    self.hop_length=hop_length #스펙트로그램에서 가져올 크기
    self._min_max_normaliser=MinMaxNormaliser(0,1)#정규화 크기는 0부터 1까지

  def generate(self,spectrograms,min_max_values): #생성
    generated_spectrograms,latent_representations=self.vae.reconstruct(spectrograms) #새로 만들 스펙트로그램, 특징벡터
    signals=self.convert_spectrograms_to_audio(generated_spectrograms,min_max_values)#해당 데이터들로 오디오 생성
    return signals,latent_representations

  def convert_spectrograms_to_audio(self,spectrograms,min_max_values):#오디오 생성 함수
    signals=[]
    for spectrograms,min_max_value in zip(spectrograms,min_max_values):
      log_spectrogram=spectrogram[:,:,0] #로그 스펙트럼 형태변환, 주기확인 용이함
      denorm_log_spec=self._min_max_normaliser.denormalise( #정규화 품
          log_spectrogram,min_max_value["min"],min_max_value["max"])#로그 스펙트럼을 일반 스펙트럼으로 바꿈
      spec=librosa.db_to_amplitude(denorm_log_spec)#데시벨->진폭
      signal=librosa.istft(spec,hop_length=self.hop_length)#역 푸리에 변환
      signals.append(signal) #웨이브파일로 변경됨
    return signals